# Работа с GigaChat Vision
В этом ноутбуке мы рассмотрим работу с GigaChat Vision

Наши действия:
1. Загружаем фото на S3 хранилище GigaChat
2. Просим GigaChat проанализировать фотографии, выделить сущности на них и написать описание к фото
3. Выдать ответ в виде JSON, после чего преобразуем этот JSON в Pydantic модель

Тестировать будем на этих фото
![фото 1](cat.jpg)
![фото 2](sea.jpg)

## Установка зависимостей

Для работы примера установите зависимости:

In [7]:
!pip install langchain-gigachat langchain python-dotenv -q


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


## Инициализация GigaChat

Для работы к моделям GigaChat, нужно передать ключ авторизации GigaChat API.

Ключ можно указать в переменной среды `GIGACHAT_CREDENTIALS`, заданной в файле `.env` или созданной с помощью команды:

```sh
export GIGACHAT_CREDENTIALS=ключ_авториазации
```

При инициализации проверяется наличие переменной среды `GIGACHAT_CREDENTIALS` с заданным ключом авторизации GigaChat API.
Если переменная отсутствует, вы сможете указать ключ в поле **Введите ключ авторизации GigaChat API**.

О способах авторизации и поддерживаемых переменных среды — в [README библиотеки gigachat](https://github.com/ai-forever/gigachat).

In [1]:
import getpass
import os
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

if "GIGACHAT_CREDENTIALS" not in os.environ:
    os.environ["GIGACHAT_CREDENTIALS"] = getpass.getpass("Введите ключ авторизации GigaChat API: ")

from langchain_gigachat.chat_models import GigaChat

llm = GigaChat(
    temperature=0.1,
    verify_ssl_certs=False,
    timeout=6000,
    model="GigaChat-Pro"
)

In [2]:
from typing import List, Optional

from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field, validator
from langchain_core.runnables import RunnableLambda, RunnableParallel


class Photo(BaseModel):
    """Информация о фото"""

    content: str = Field(..., description="Что изображено на фото? 1-3 слова")
    description: str = Field(..., description="Опиши детальнее фото")


# Set up a parser
parser = PydanticOutputParser(pydantic_object=Photo)


def _get_messages_from_url(url: str):
    return {
        "history": [
            HumanMessage(content="", additional_kwargs={"attachments": [url]}),
        ]
    }


# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Определи объект на фото. Ответь на запрос пользователя в формате JSON. Schema Information: \n{format_instructions}",
        ),
        MessagesPlaceholder("history"),
    ]
).partial(format_instructions=parser.get_format_instructions())

chain = RunnableLambda(_get_messages_from_url) | prompt | llm | parser

/Users/18673816/Documents/GIGACHAIN_TRUE/gigachain/.venv/lib/python3.11/site-packages/langchain/output_parsers/__init__.py:33: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain.output_parsers.combining import CombiningOutputParser


В итоге мы создали LCEL цепочку, в которую мы можем передавать id загруженных файлов и в ответ получать Pydantic модели

In [3]:
# Пример работы
file = llm.upload_file(open("cat.jpg", "rb"))
file2 = llm.upload_file(open("sea.jpg", "rb"))
chain.batch([file.id_, file2.id_])

[Photo(content='котенок', description='милый котенок с черно-белой шерстью и большими глазами'),
 Photo(content='пляж', description='Пляж с каменистым берегом и волнами у моря.')]